In [255]:
# import all necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

## Data Wrangling

In [256]:
# Common functions
def print_series(x):
    for i in range(len(x)):
        print(i, ': ', x[i])
        
def get_tech_dummies(x):
    x = x.notna()
    x = x.apply(lambda y: int(1) if y == True else int(0))
    return x


In [257]:
# Functions to filter and clean up columns 
def clean_filter_cols(df, non_code_cols, non_code_col_names, code_cols, 
                          code_col_name_from_subheader=False, 
                          code_col_dummies_sep=None):

    # store subheader row for later use and drop it
    if code_col_name_from_subheader:
        subheaders = df.iloc[0]
        df = df.drop(index=0)
    
    # prepare non_code dataframe
    non_code_df = df.filter(df.columns[non_code_cols])
    non_code_df.columns = non_code_col_names
    
    # prepare code dataframe
    code_df = df.filter(df.columns[code_cols])
    
    # handle naming of column from subheader row
    if code_col_name_from_subheader:
#         code_subheaders = code_df.iloc[0]
        code_df.columns = list(subheaders.filter(df.columns[code_cols]))
            
    # convert code cols to dummies
    if code_col_dummies_sep is None:
        code_df = code_df.apply(lambda x: get_tech_dummies(x))
    else:
        dummies_df = pd.DataFrame()
        for col in code_df.columns:
            dummies_df = pd.concat([dummies_df, code_df[col].str.get_dummies(sep=code_col_dummies_sep)], axis=1)
        code_df = dummies_df
        
    # convert all column names to lower case
    code_df.columns = code_df.columns.str.lower() 
    
    # combine the data
    new_df = pd.concat([non_code_df, code_df], axis=1)
    
    return new_df

In [259]:
# prepare salary ordered type
def categorize_salary_range(sal):
    
    salary_map = {'<$20,000': 'Less than $20,000', 
                  '>$140,000': 'More than $140,000', 
                  'Less than $10,000': 'Less than $20,000',
                  'Unemployed': 'Student / Unemployed',
                  '$140,000 - $160,000': 'More than $140,000',
                  'More than $160,000': 'More than $140,000'}
    
    sal = sal.apply(lambda x : salary_map.get(x, x))
    
    salary_order = ['Student / Unemployed', 
                    'Less than $20,000', 
                    '$20,000 - $40,000', 
                    '$40,000 - $60,000', 
                    '$60,000 - $80,000', 
                    '$80,000 - $100,000', 
                    '$100,000 - $120,000',
                    '$120,000 - $140,000', 
                    'More than $140,000']
    
    salary_type = pd.api.types.CategoricalDtype(salary_order, ordered=True)
    sal = sal.astype(salary_type)
    
    return sal


# split the salary values to bins
def get_salary_bins(sal):
    bins = [0, 20000, 40000, 60000, 80000, 100000, 120000, 140000, 2250000]
    labels = ['Less than $20,000', '$20,000 - $40,000', '$40,000 - $60,000', '$60,000 - $80,000', '$80,000 - $100,000',
         '$100,000 - $120,000', '$120,000 - $140,000', 'More than $140,000']
    sal = pd.cut(sal, bins, right=True, labels=labels)
    return sal


# print the counts and mean counts of each salary range
def print_salary_values(sal):
    df_means = (sal.value_counts() / sal.shape[0])
    df_counts = sal.value_counts()
    df_sal = pd.concat([df_means, df_counts], axis=1)
    df_sal.columns = ['mean', 'count']
    df_sal = df_sal.sort_values(by='mean')
    
    return df_sal



### read and process 2011 data

In [260]:
df_2011 = pd.read_csv('2011_results.csv', encoding='latin_1')
df_2011.head()

,What Country or Region do you live in?,Which US State or Territory do you live in?,How old are you?,How many years of IT/Programming experience do you have?,How would you best describe the industry you work in?,Which best describes the size of your company?,Which of the following best describes your occupation?,How likely is it that a recommendation you make will be acted upon?,What is your involvement in purchasing? You can choose more than 1.,Unnamed: 9,...,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,"In the last 12 months, how much money have you spent on personal technology-related purchases?",Which of our sites do you frequent most?
0,Response,Response,Response,Response,Response,Response,Response,Response,Influencer,Recommender,...,AppleTV,iPad,Other netbook,PS3,Xbox,Wii,Other gaming system,other (please specify),Response,Response
1,Africa,NaN,< 20,<2,Consulting,Start Up (1-25),Web Application Developer,Not in a million years,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<$100,NaN
2,Other Europe,NaN,25-29,41310,Software Products,Mature Small Business (25-100),Server Programmer,It's been known to happen,NaN,NaN,...,NaN,NaN,Other netbook,NaN,NaN,NaN,NaN,NaN,$251-$500,Stack Overflow
3,India,NaN,25-29,41435,Software Products,Mid Sized (100-999),Server Programmer,Unless it's stoopid it gets done,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Germany,NaN,< 20,41310,Foundation / Non-Profit,Student,Student,It's been known to happen,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Wii,Other gaming system,NaN,"$501-$1,000",Stack Overflow


In [261]:
print_series(df_2011.columns)

0 :  What Country or Region do you live in?
1 :  Which US State or Territory do you live in?
2 :  How old are you?
3 :  How many years of IT/Programming experience do you have?
4 :  How would you best describe the industry you work in?
5 :  Which best describes the size of your company?
6 :  Which of the following best describes your occupation?
7 :  How likely is it that a recommendation you make will be acted upon?
8 :  What is your involvement in purchasing? You can choose more than 1.
9 :  Unnamed: 9
10 :  Unnamed: 10
11 :  Unnamed: 11
12 :  Unnamed: 12
13 :  Unnamed: 13
14 :  Unnamed: 14
15 :  What types of purchases are you involved in?
16 :  Unnamed: 16
17 :  Unnamed: 17
18 :  Unnamed: 18
19 :  Unnamed: 19
20 :  Unnamed: 20
21 :  What is your budget for outside expenditures (hardware, software, consulting, etc) for 2011?
22 :  Unnamed: 22
23 :  Unnamed: 23
24 :  Unnamed: 24
25 :  Unnamed: 25
26 :  Unnamed: 26
27 :  Unnamed: 27
28 :  Unnamed: 28
29 :  What type of project are you

In [262]:
print_series(df_2011.iloc[0])

0 :  Response
1 :  Response
2 :  Response
3 :  Response
4 :  Response
5 :  Response
6 :  Response
7 :  Response
8 :  Influencer
9 :  Recommender
10 :  Approver
11 :  Purchaser
12 :  Check Writer
13 :  No Involvement
14 :  I'm a Seller
15 :  Hardware
16 :  Servers
17 :  Software
18 :  User Equipment: Monitors, PCs, Laptops
19 :  Consultants
20 :  Other
21 :  <$10,000
22 :  $10,001 - $25,000
23 :  $25,001 - $40,000
24 :  $41,000 - $75,000
25 :  $75,001 - $100,000
26 :  $100,001 - $150,000
27 :  >$150,000
28 :  Don't know
29 :  Response
30 :  Java
31 :  JavaScript
32 :  CSS
33 :  PHP
34 :  Python
35 :  Ruby
36 :  SQL
37 :  C#
38 :  C++
39 :  C
40 :  Perl
41 :  None
42 :  other (please specify)
43 :  Response
44 :  Response
45 :  Response
46 :  iPhone
47 :  Android
48 :  Blackberry
49 :  Other Smart Phone
50 :  Regular Mobile Phone
51 :  Kindle
52 :  Nook
53 :  Blu-Ray
54 :  HDTV
55 :  AppleTV
56 :  iPad
57 :  Other netbook
58 :  PS3
59 :  Xbox
60 :  Wii
61 :  Other gaming system
62 :  oth

In [263]:
# extract the columns we are interested in and the tech cols
df_2011 = clean_filter_cols(df_2011, [2,3,45], ['age', 'experience', 'salary'], list(np.arange(30,42)), True)
df_2011.head()

,age,experience,salary,java,javascript,css,php,python,ruby,sql,c#,c++,c,perl,none
1,< 20,<2,Student / Unemployed,0,1,0,0,0,0,0,0,0,0,0,0
2,25-29,41310,NaN,1,0,0,0,0,0,1,0,0,1,0,0
3,25-29,41435,NaN,1,1,0,0,0,0,1,0,0,0,0,0
4,< 20,41310,Student / Unemployed,1,0,0,0,0,0,0,0,0,0,0,0
5,35-39,11,"$80,000 - $100,000",1,1,1,1,0,0,1,0,1,1,1,0


In [264]:
# no need to consider those who don't program in any of the mainstream languages
df_2011 = df_2011.drop(columns=['none'])

In [265]:
# add year marker to the data
df_2011.insert(0,'year',2011)

In [266]:
# add gender column to data which will come in later datasets
df_2011.insert(3, 'gender', np.nan)

In [267]:
# convert salary to ordered slabs
df_2011.salary = categorize_salary_range(df_2011.salary)

In [268]:
# check salary values
print_salary_values(df_2011.salary)

,mean,count
"$120,000 - $140,000",0.034483,97
"More than $140,000",0.039104,110
Student / Unemployed,0.076431,215
"$100,000 - $120,000",0.078919,222
"Less than $20,000",0.100604,283
"$80,000 - $100,000",0.103804,292
"$20,000 - $40,000",0.113047,318
"$60,000 - $80,000",0.146818,413
"$40,000 - $60,000",0.147529,415


In [269]:
df_2011.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2813 entries, 1 to 2813
Data columns (total 16 columns):
year          2813 non-null int64
age           2729 non-null object
experience    2729 non-null object
gender        0 non-null float64
salary        2365 non-null category
java          2813 non-null int64
javascript    2813 non-null int64
css           2813 non-null int64
php           2813 non-null int64
python        2813 non-null int64
ruby          2813 non-null int64
sql           2813 non-null int64
c#            2813 non-null int64
c++           2813 non-null int64
c             2813 non-null int64
perl          2813 non-null int64
dtypes: category(1), float64(1), int64(12), object(2)
memory usage: 354.8+ KB


In [270]:
df_2011.describe()

,year,gender,java,javascript,css,php,python,ruby,sql,c#,c++,c,perl
count,2813.0,0.0,2813.000000,2813.000000,2813.000000,2813.000000,2813.000000,2813.000000,2813.000000,2813.000000,2813.000000,2813.000000,2813.000000
mean,2011.0,NaN,0.306079,0.504444,0.482048,0.280128,0.204408,0.101315,0.573409,0.480626,0.263064,0.263420,0.100960
std,0.0,NaN,0.460945,0.500069,0.499766,0.449141,0.403340,0.301799,0.494670,0.499713,0.440375,0.440566,0.301329
min,2011.0,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2011.0,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2011.0,NaN,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
75%,2011.0,NaN,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000
max,2011.0,NaN,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### read and process 2012 data

In [271]:
df_2012 = pd.read_csv('2012_results.csv', encoding='latin_1')
df_2012.head()

,What Country or Region do you live in?,Which US State or Territory do you live in?,How old are you?,How many years of IT/Programming experience do you have?,How would you best describe the industry you currently work in?,Which best describes the size of your company?,Which of the following best describes your occupation?,What is your involvement in purchasing products or services for the company you work for? (You can choose more than one),Unnamed: 8,Unnamed: 9,...,Please rate the advertising you've seen on Stack Overflow,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,What advertisers do you remember seeing on Stack Overflow?,What is your current Stack Overflow reputation?,Which of our sites do you frequent most?,Unnamed: 74
0,Response,Response,Response,Response,Response,Response,Response,Influencer,Recommender,Approver,...,The ads are relevant,The ads are entertaining,The ads are Informative,I click on ads that interest me,I've taken a trial/purchased a product from ads,What ads? I use an ad blocker,Open-Ended Response,Response,Response,Other Stack Exchange (please specify)
1,India,NaN,20-24,<2,Consulting,"Fortune 1000 (1,000+)",Server Programmer,Influencer,Recommender,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Germany,NaN,25-29,<2,Other,Mature Small Business (25-100),Embedded Application Developer,NaN,Recommender,NaN,...,Neutral,Agree,Agree,Strongly Disagree,Strongly Disagree,Disagree,NaN,Don't have an account,Stack Overflow,NaN
3,United Kingdom,NaN,20-24,41070,Finance / Banking,Mature Small Business (25-100),Web Application Developer,Influencer,Recommender,NaN,...,Neutral,Neutral,Neutral,Neutral,Neutral,Strongly Agree,NaN,1,Stack Overflow,NaN
4,France,NaN,20-24,40944,Software Products,Mature Small Business (25-100),Embedded Application Developer,NaN,NaN,NaN,...,Agree,Strongly Disagree,Disagree,Disagree,Strongly Disagree,Disagree,None !,Don't have an account,Stack Overflow,NaN


In [272]:
print_series(df_2012.columns)

0 :  What Country or Region do you live in?
1 :  Which US State or Territory do you live in?
2 :  How old are you?
3 :  How many years of IT/Programming experience do you have?
4 :  How would you best describe the industry you currently work in?
5 :  Which best describes the size of your company?
6 :  Which of the following best describes your occupation?
7 :  What is your involvement in purchasing products or services for the company you work for? (You can choose more than one)
8 :  Unnamed: 8
9 :  Unnamed: 9
10 :  Unnamed: 10
11 :  Unnamed: 11
12 :  Unnamed: 12
13 :  Unnamed: 13
14 :  What types of purchases are you involved in?
15 :  Unnamed: 15
16 :  Unnamed: 16
17 :  Unnamed: 17
18 :  Unnamed: 18
19 :  Unnamed: 19
20 :  What is your budget for outside expenditures (hardware, software, consulting, etc) for 2011?
21 :  What type of project are you developing?
22 :  Which languages are you proficient in?
23 :  Unnamed: 23
24 :  Unnamed: 24
25 :  Unnamed: 25
26 :  Unnamed: 26
27 :  Un

In [273]:
print_series(df_2012.iloc[0])

0 :  Response
1 :  Response
2 :  Response
3 :  Response
4 :  Response
5 :  Response
6 :  Response
7 :  Influencer
8 :  Recommender
9 :  Approver
10 :  Purchaser
11 :  Check Writer
12 :  No Involvement
13 :  I'm a Seller
14 :  Hardware
15 :  Servers
16 :  Software
17 :  User Equipment: Monitors, PCs, Laptops
18 :  Consultants
19 :  Other
20 :  Response
21 :  Response
22 :  Java
23 :  JavaScript
24 :  CSS
25 :  PHP
26 :  Python
27 :  Objective-C
28 :  Ruby
29 :  SQL
30 :  C#
31 :  C++
32 :  C
33 :  Perl
34 :  HTML5
35 :  None
36 :  Other (please specify)
37 :  Response
38 :  Response
39 :  Response
40 :  Response
41 :  Response
42 :  Response
43 :  Other (please specify)
44 :  iPhone
45 :  Android
46 :  Blackberry
47 :  Windows Phone
48 :  Other Smart Phone
49 :  Regular Mobile Phone
50 :  Kindle
51 :  Nook
52 :  AppleTV
53 :  Boxee
54 :  Other media streaming device
55 :  Netbook
56 :  PS3
57 :  Xbox
58 :  Wii
59 :  Other gaming system
60 :  Kindle Fire
61 :  iPad
62 :  Other tablet
63 

In [274]:
# extract the columns we are interested in and the tech cols
df_2012 = clean_filter_cols(df_2012, [2,3,39], ['age', 'experience', 'salary'], list(np.arange(22,36)), True)
df_2012.head()

,age,experience,salary,java,javascript,css,php,python,objective-c,ruby,sql,c#,c++,c,perl,html5,none
1,20-24,<2,"<$20,000",1,1,0,0,0,0,0,1,0,1,1,0,0,0
2,25-29,<2,"$20,000 - $40,000",0,1,1,1,0,1,0,0,0,1,0,0,1,0
3,20-24,41070,"$20,000 - $40,000",0,0,1,1,0,1,0,1,0,0,0,0,1,0
4,20-24,40944,"$20,000 - $40,000",1,0,0,0,0,0,0,0,0,1,0,0,0,0
5,< 20,40944,Student / Unemployed,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [275]:
# ignore columns where language choice is none
df_2012 = df_2012.drop(columns=['none'])

In [276]:
# consistent column names
df_2012 = df_2012.rename(columns={'html5':'html'})

In [277]:
# insert year marker for data
df_2012.insert(0,'year',2012)

In [278]:
# convert salary to ordered slabs
df_2012.salary = categorize_salary_range(df_2012.salary)

In [279]:
# check salary values
print_salary_values(df_2012.salary)

,mean,count
"$120,000 - $140,000",0.023546,147
"More than $140,000",0.028992,181
"$100,000 - $120,000",0.047733,298
"$80,000 - $100,000",0.076726,479
Student / Unemployed,0.082492,515
"$60,000 - $80,000",0.097549,609
"$20,000 - $40,000",0.103636,647
"Less than $20,000",0.110203,688
"$40,000 - $60,000",0.110684,691


In [280]:
df_2012.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6243 entries, 1 to 6243
Data columns (total 17 columns):
year           6243 non-null int64
age            5964 non-null object
experience     5964 non-null object
salary         4255 non-null category
java           6243 non-null int64
javascript     6243 non-null int64
css            6243 non-null int64
php            6243 non-null int64
python         6243 non-null int64
objective-c    6243 non-null int64
ruby           6243 non-null int64
sql            6243 non-null int64
c#             6243 non-null int64
c++            6243 non-null int64
c              6243 non-null int64
perl           6243 non-null int64
html           6243 non-null int64
dtypes: category(1), int64(14), object(2)
memory usage: 835.6+ KB


In [281]:
df_2012.describe()

,year,java,javascript,css,php,python,objective-c,ruby,sql,c#,c++,c,perl,html
count,6243.0,6243.000000,6243.000000,6243.000000,6243.000000,6243.000000,6243.000000,6243.000000,6243.000000,6243.000000,6243.000000,6243.000000,6243.000000,6243.000000
mean,2012.0,0.376101,0.500881,0.452347,0.308345,0.193817,0.096909,0.092424,0.527631,0.395323,0.244434,0.237706,0.073843,0.345026
std,0.0,0.484445,0.500039,0.497764,0.461847,0.395319,0.295857,0.289646,0.499276,0.488959,0.429785,0.425712,0.261536,0.475415
min,2012.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2012.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2012.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2012.0,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000
max,2012.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### read and process 2013 data

In [282]:
df_2013 = pd.read_csv('2013_results.csv', encoding='latin_1', low_memory=False)
df_2013.head()

,What Country or Region do you live in?,Which US State or Territory do you live in?,How old are you?,How many years of IT/Programming experience do you have?,How would you best describe the industry you currently work in?,How many people work for your company?,Which of the following best describes your occupation?,"Including yourself, how many developers are employed at your company?",How large is the team that you work on?,What other departments / roles do you interact with regularly?,...,Unnamed: 118,Unnamed: 119,Unnamed: 120,Unnamed: 121,What advertisers do you remember seeing on Stack Overflow?,What is your current Stack Overflow reputation?,How do you use Stack Overflow?,Unnamed: 125,Unnamed: 126,Unnamed: 127
0,Response,Response,Response,Response,Response,Response,Response,Response,Response,System Administrators,...,The ads are Informative,I click on ads that interest me,I've taken a trial/purchased a product from ads,What ads? I use an ad blocker,Open-Ended Response,Response,Read other people's questions to solve my prob...,Ask questions to solve problems,Answer questions I know the answer to,Other (please specify)
1,United Kingdom,NaN,35-39,6/10/2013,Finance / Banking,101-999,Enterprise Level Services,100,4/8/2013,System Administrators,...,Neutral,Neutral,Neutral,Neutral,NaN,Don't have an account,Read other people's questions to solve my prob...,NaN,NaN,NaN
2,United States of America,Oregon,25-29,6/10/2013,Retail,101-999,Back-End Web Developer,6/15/2013,4/8/2013,System Administrators,...,Neutral,Agree,Disagree,Neutral,"StackOverflow themselves, Careers 2.0 (SO also...",1,Read other people's questions to solve my prob...,Ask questions to solve problems,Answer questions I know the answer to,NaN
3,United States of America,Wisconsin,51-60,11,Software Products,26-100,Enterprise Level Services,6/15/2013,Just me!,System Administrators,...,Neutral,Strongly Disagree,Strongly Disagree,Strongly Disagree,don't recall seeing ads on Stack Overflow,Don't have an account,Read other people's questions to solve my prob...,NaN,NaN,NaN
4,Germany,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [283]:
# merge JQuery and jQuery columns
df_2013['Unnamed: 61'] = df_2013['Unnamed: 61'].where(df_2013['Unnamed: 61'].notna(), df_2013['Unnamed: 62'])
df_2013 = df_2013.drop(columns=['Unnamed: 62'])

In [284]:
print_series(df_2013.columns)

0 :  What Country or Region do you live in?
1 :  Which US State or Territory do you live in?
2 :  How old are you?
3 :  How many years of IT/Programming experience do you have?
4 :  How would you best describe the industry you currently work in?
5 :  How many people work for your company?
6 :  Which of the following best describes your occupation?
7 :  Including yourself, how many developers are employed at your company?
8 :  How large is the team that you work on?
9 :  What other departments / roles do you interact with regularly?
10 :  Unnamed: 10
11 :  Unnamed: 11
12 :  Unnamed: 12
13 :  Unnamed: 13
14 :  Unnamed: 14
15 :  Unnamed: 15
16 :  Unnamed: 16
17 :  Unnamed: 17
18 :  Unnamed: 18
19 :  If your company has a native mobile app, what platforms do you support?
20 :  Unnamed: 20
21 :  Unnamed: 21
22 :  Unnamed: 22
23 :  Unnamed: 23
24 :  Unnamed: 24
25 :  Unnamed: 25
26 :  If you make a software product, how does your company make money? (You can choose more than one)
27 :  Unnam

In [285]:
print_series(df_2013.iloc[0])

0 :  Response
1 :  Response
2 :  Response
3 :  Response
4 :  Response
5 :  Response
6 :  Response
7 :  Response
8 :  Response
9 :  System Administrators
10 :  Designers
11 :  Product Managers
12 :  Testers / Quality Assurance
13 :  Technical Support
14 :  Sales / Marketing
15 :  Consultants
16 :  Customers
17 :  Finance
18 :  Human Resources
19 :  iPhone
20 :  iPad
21 :  Android phone
22 :  Android tablet
23 :  Blackberry
24 :  Other
25 :  No mobile app
26 :  Advertising
27 :  Direct sales to consumers
28 :  Direct sales to companies
29 :  Software as a service / recurring billing
30 :  Mobile app sales
31 :  Consulting
32 :  Grants / outside fund-raising
33 :  Other
34 :  Developing new features
35 :  Refactoring / code quality
36 :  Fixing bugs
37 :  Technical support
38 :  Meetings
39 :  Learning new skills
40 :  Surfing the Internet
41 :  Commuting
42 :  I can recommend or request products
43 :  I influence purchasing decisions, but don't have final approval
44 :  I have a discreti

In [286]:
# extract the columns we are interested in and the tech cols
df_2013 = clean_filter_cols(df_2013, [2,3,99], ['age','experience','salary'], list(np.arange(56,68)), True)
df_2013.head()

,age,experience,salary,c,c++,c#,java,javascript,jquery,node.js,objective-c,php,python,ruby,sql
1,35-39,6/10/2013,"$80,000 - $100,000",0,0,0,1,0,0,0,0,0,0,0,1
2,25-29,6/10/2013,"$20,000 - $40,000",0,0,1,0,1,1,0,0,1,0,0,0
3,51-60,11,"$120,000 - $140,000",0,0,1,0,1,1,0,0,0,0,0,1
4,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0
5,35-39,11,NaN,0,0,0,0,0,0,0,0,0,0,0,0


In [287]:
# insert year marker for data
df_2013.insert(0,'year',2013)

In [288]:
# convert salary to ordered slabs
df_2013.salary = categorize_salary_range(df_2013.salary)

In [289]:
# check salary values
print_salary_values(df_2013.salary)

,mean,count
"$120,000 - $140,000",0.026073,254
"More than $140,000",0.033155,323
"$100,000 - $120,000",0.054198,528
Student / Unemployed,0.058510,570
"$80,000 - $100,000",0.074625,727
"Less than $20,000",0.078731,767
"$20,000 - $40,000",0.088688,864
"$40,000 - $60,000",0.093513,911
"$60,000 - $80,000",0.101211,986


In [290]:
df_2013.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9742 entries, 1 to 9742
Data columns (total 16 columns):
year           9742 non-null int64
age            9434 non-null object
experience     9436 non-null object
salary         5930 non-null category
c              9742 non-null int64
c++            9742 non-null int64
c#             9742 non-null int64
java           9742 non-null int64
javascript     9742 non-null int64
jquery         9742 non-null int64
node.js        9742 non-null int64
objective-c    9742 non-null int64
php            9742 non-null int64
python         9742 non-null int64
ruby           9742 non-null int64
sql            9742 non-null int64
dtypes: category(1), int64(13), object(2)
memory usage: 1.2+ MB


In [291]:
df_2013.describe()

,year,c,c++,c#,java,javascript,jquery,node.js,objective-c,php,python,ruby,sql
count,9742.0,9742.000000,9742.000000,9742.000000,9742.000000,9742.000000,9742.000000,9742.000000,9742.000000,9742.000000,9742.000000,9742.000000,9742.000000
mean,2013.0,0.147403,0.174092,0.310716,0.309998,0.486040,0.425888,0.061486,0.098029,0.238555,0.192979,0.081503,0.471464
std,0.0,0.354525,0.379208,0.462811,0.462516,0.499831,0.494502,0.240233,0.297369,0.426222,0.394657,0.273620,0.499211
min,2013.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2013.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2013.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2013.0,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,2013.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### Read and process 2014 data

In [292]:
df_2014 = pd.read_csv('2014_results.csv')
df_2014.head()

,What Country do you live in?,Unnamed: 1,Which US State or Territory do you live in?,How old are you?,What is your gender?,How many years of IT/Programming experience do you have?,Which of the following best describes your occupation?,"Including bonus, what is your annual compensation in USD?",How would you best describe the industry you currently work in?,How many developers are employed at your company?,...,Did you participate in the Apptivate contest?,What advertisers do you remember seeing on Stack Overflow?,What is your current Stack Overflow reputation?,How do you use Stack Overflow?,Unnamed: 114,Unnamed: 115,Unnamed: 116,Unnamed: 117,Unnamed: 118,How often do you find solutions to your programming problems on Stack Overflow without asking a new question?
0,Response,Other (please specify),Response,Response,Response,Response,Response,Response,Response,Response,...,Response,Open-Ended Response,Response,Read other people's questions to solve my prob...,Ask questions to solve problems,Answer questions I know the answer to,Looking for a job,Build my online reputation,Other (please specify),Response
1,India,NaN,NaN,30-34,Female,6/10/2014,Back-End Web Developer,"$20,000 - $40,000",Finance / Banking,100,...,No,NaN,500,Read other people's questions to solve my prob...,Ask questions to solve problems,NaN,NaN,NaN,NaN,Almost Always
2,Thailand,NaN,NaN,20-24,Male,<2,Back-End Web Developer,Student / Unemployed,Healthcare,NaN,...,NaN,NaN,Don't have an account,Read other people's questions to solve my prob...,NaN,NaN,NaN,NaN,NaN,NaN
3,Iran,NaN,NaN,25-29,Male,6/10/2014,Desktop Software Developer,"<$20,000",Not Currently Employed,1/5/2014,...,No,Tehcodez,1,Read other people's questions to solve my prob...,NaN,NaN,NaN,NaN,NaN,Almost Always
4,Ukraine,NaN,NaN,< 20,Male,<2,Student,Student / Unemployed,Student,NaN,...,NaN,NaN,50,Read other people's questions to solve my prob...,Ask questions to solve problems,NaN,NaN,NaN,NaN,Almost Always


In [293]:
print_series(df_2014.columns)

0 :  What Country do you live in?
1 :  Unnamed: 1
2 :  Which US State or Territory do you live in?
3 :  How old are you?
4 :  What is your gender?
5 :  How many years of IT/Programming experience do you have?
6 :  Which of the following best describes your occupation?
7 :  Including bonus, what is your annual compensation in USD?
8 :  How would you best describe the industry you currently work in?
9 :  How many developers are employed at your company?
10 :  Do you work remotely?
11 :  Do you enjoy working remotely?
12 :  Where do you work remotely most of the time?
13 :  If your company has a native mobile app, what platforms do you support?
14 :  Unnamed: 14
15 :  Unnamed: 15
16 :  Unnamed: 16
17 :  Unnamed: 17
18 :  Unnamed: 18
19 :  Unnamed: 19
20 :  In an average week, how do you spend your time at work?
21 :  Unnamed: 21
22 :  Unnamed: 22
23 :  Unnamed: 23
24 :  Unnamed: 24
25 :  Unnamed: 25
26 :  Unnamed: 26
27 :  Unnamed: 27
28 :  Unnamed: 28
29 :  What is your involvement in pu

In [294]:
print_series(df_2014.iloc[0])

0 :  Response
1 :  Other (please specify)
2 :  Response
3 :  Response
4 :  Response
5 :  Response
6 :  Response
7 :  Response
8 :  Response
9 :  Response
10 :  Response
11 :  Response
12 :  Response
13 :  iPhone
14 :  iPad
15 :  Android Phone
16 :  Android Tablet
17 :  Windows Phone
18 :  Other
19 :  No mobile app
20 :  Developing new features
21 :  Refactoring / code quality
22 :  Fixing bugs
23 :  Technical support
24 :  Meetings
25 :  Learning new skills
26 :  Surfing the Internet
27 :  Looking for a new job
28 :  Using Stack Exchange
29 :  I can recommend or request products
30 :  I influence purchasing decisions, but don't have final approval
31 :  I have a discretionary budget at my disposal
32 :  I can buy anything I want without asking anyone
33 :  No Involvement
34 :  Hardware
35 :  Servers
36 :  Software
37 :  User Equipment: Monitors, PCs, Laptops
38 :  Consultants
39 :  Recruitment Tools & Services
40 :  Other
41 :  Response
42 :  C
43 :  C++
44 :  C#
45 :  Java
46 :  JavaS

In [295]:
# extract the columns we are interested in and the tech cols
df_2014 = clean_filter_cols(df_2014, [3,4,5,7], ['age','gender','experience', 'salary'], list(np.arange(42,53)), True)
df_2014.head()

,age,gender,experience,salary,c,c++,c#,java,javascript,node.js,objective-c,php,python,ruby,sql
1,30-34,Female,6/10/2014,"$20,000 - $40,000",0,0,0,1,1,0,0,1,1,0,0
2,20-24,Male,<2,Student / Unemployed,0,0,0,0,0,0,0,1,0,0,0
3,25-29,Male,6/10/2014,"<$20,000",0,0,1,0,1,0,0,0,0,0,1
4,< 20,Male,<2,Student / Unemployed,0,1,0,0,0,0,0,0,0,0,0
5,25-29,Male,2/5/2014,Rather not say,0,0,0,1,1,0,0,0,0,0,1


In [296]:
# insert year marker for data
df_2014.insert(0,'year',2014)

In [297]:
# convert salary to ordered slabs
df_2014.salary = categorize_salary_range(df_2014.salary)

In [298]:
# check salary values
print_salary_values(df_2014.salary)

,mean,count
"$120,000 - $140,000",0.026037,199
"More than $140,000",0.031925,244
"$100,000 - $120,000",0.050504,386
"$80,000 - $100,000",0.076802,587
"$60,000 - $80,000",0.091195,697
"$20,000 - $40,000",0.097606,746
"$40,000 - $60,000",0.103886,794
"Less than $20,000",0.126914,970
Student / Unemployed,0.150072,1147


In [299]:
df_2014.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7643 entries, 1 to 7643
Data columns (total 16 columns):
year           7643 non-null int64
age            7346 non-null object
gender         7346 non-null object
experience     7346 non-null object
salary         5770 non-null category
c              7643 non-null int64
c++            7643 non-null int64
c#             7643 non-null int64
java           7643 non-null int64
javascript     7643 non-null int64
node.js        7643 non-null int64
objective-c    7643 non-null int64
php            7643 non-null int64
python         7643 non-null int64
ruby           7643 non-null int64
sql            7643 non-null int64
dtypes: category(1), int64(12), object(3)
memory usage: 963.2+ KB


In [300]:
df_2014.describe()

,year,c,c++,c#,java,javascript,node.js,objective-c,php,python,ruby,sql
count,7643.0,7643.000000,7643.000000,7643.000000,7643.000000,7643.000000,7643.000000,7643.000000,7643.000000,7643.000000,7643.000000,7643.000000
mean,2014.0,0.153605,0.185529,0.311134,0.324349,0.494963,0.083868,0.088316,0.223734,0.192725,0.077587,0.453618
std,0.0,0.360593,0.388752,0.462988,0.468162,0.500007,0.277207,0.283773,0.416773,0.394465,0.267539,0.497877
min,2014.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2014.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2014.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2014.0,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,2014.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### Read and process 2015 data

In [301]:
df_2015 = pd.read_csv('2015_results.csv', skiprows=1, low_memory=False)
df_2015.head()

,Country,Age,Gender,Tabs or Spaces,Years IT / Programming Experience,Occupation,Desktop Operating System,Desktop Operating System: write-in,Current Lang & Tech: Android,Current Lang & Tech: Arduino,...,Why use Stack Overflow: I don't use Stack Overflow,How often are Stack Overflow's answers helpful,Why answer: Help a programmer in need,Why answer: Help future programmers,Why answer: Demonstrate expertise,Why answer: Self promotion,Why answer: Sense of responsibility to developers,Why answer: No idea,Why answer: I don't answer and I don't want to,Why answer: I don't answer but I want to
0,Croatia,25-29,Male,Tabs,2 - 5 years,Back-end web developer,Ubuntu,NaN,NaN,NaN,...,NaN,Usually,NaN,NaN,It feels good to demonstrate my expertise.,Demonstrating my expertise will benefit me,I feel a sense of responsibility to the develo...,NaN,NaN,NaN
1,France,20-24,Male,Spaces,1 - 2 years,Back-end web developer,Windows 7,NaN,NaN,NaN,...,NaN,Usually,NaN,My answer will help lots of people who have th...,It feels good to demonstrate my expertise.,NaN,NaN,NaN,NaN,NaN
2,India,20-24,Male,Tabs,1 - 2 years,Back-end web developer,Windows 7,NaN,NaN,NaN,...,NaN,Rarely,NaN,NaN,NaN,Demonstrating my expertise will benefit me,NaN,NaN,NaN,NaN
3,Latvia,25-29,Male,It depends,6 - 10 years,Back-end web developer,Ubuntu,NaN,NaN,NaN,...,NaN,Usually,It feels good to help a programmer in need,My answer will help lots of people who have th...,It feels good to demonstrate my expertise.,Demonstrating my expertise will benefit me,I feel a sense of responsibility to the develo...,NaN,NaN,NaN
4,Norway,30-34,Male,Tabs,2 - 5 years,Back-end web developer,Windows 8,NaN,NaN,NaN,...,NaN,Usually,It feels good to help a programmer in need,NaN,NaN,Demonstrating my expertise will benefit me,NaN,NaN,NaN,"I don't answer much (or at all), but I want to..."


In [302]:
# Rename columns
df_2015 = df_2015.rename(mapper = lambda x: x.split(':')[1].strip() if 'Current Lang & Tech' in x else x, axis=1)

In [303]:
print_series(df_2015.columns)

0 :  Country
1 :  Age
2 :  Gender
3 :  Tabs or Spaces
4 :  Years IT / Programming Experience
5 :  Occupation
6 :  Desktop Operating System
7 :  Desktop Operating System: write-in
8 :  Android
9 :  Arduino
10 :  AngularJS
11 :  C
12 :  C++
13 :  C++11
14 :  C#
15 :  Cassandra
16 :  CoffeeScript
17 :  Cordova
18 :  Clojure
19 :  Cloud
20 :  Dart
21 :  F#
22 :  Go
23 :  Hadoop
24 :  Haskell
25 :  iOS
26 :  Java
27 :  JavaScript
28 :  LAMP
29 :  Matlab
30 :  MongoDB
31 :  Node.js
32 :  Objective-C
33 :  Perl
34 :  PHP
35 :  Python
36 :  R
37 :  Redis
38 :  Ruby
39 :  Rust
40 :  Salesforce
41 :  Scala
42 :  Sharepoint
43 :  Spark
44 :  SQL
45 :  SQL Server
46 :  Swift
47 :  Visual Basic
48 :  Windows Phone
49 :  Wordpress
50 :  Write-In
51 :  Future Lang & Tech: Android
52 :  Future Lang & Tech: Arduino
53 :  Future Lang & Tech: AngularJS
54 :  Future Lang & Tech: C
55 :  Future Lang & Tech: C++
56 :  Future Lang & Tech: C++11
57 :  Future Lang & Tech: C#
58 :  Future Lang & Tech: Cassandra

In [304]:
# extract the columns we are interested in and the tech cols
df_2015 = clean_filter_cols(df_2015, [1,2,4,105], ['age','gender','experience', 'salary'], list(np.arange(8,50)))
df_2015.head()

,age,gender,experience,salary,android,arduino,angularjs,c,c++,c++11,...,salesforce,scala,sharepoint,spark,sql,sql server,swift,visual basic,windows phone,wordpress
0,25-29,Male,2 - 5 years,"$20,000 - $40,000",0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,20-24,Male,1 - 2 years,"$20,000 - $40,000",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,20-24,Male,1 - 2 years,"$80,000 - $100,000",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,25-29,Male,6 - 10 years,NaN,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,30-34,Male,2 - 5 years,"$60,000 - $80,000",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [305]:
# merge similar languages / columns
df_2015['c++'] = df_2015['c++'].where(df_2015['c++'] == 1, df_2015['c++11'])
df_2015 = df_2015.drop(columns=['c++11'])

In [306]:
# consistent column names
df_2015 = df_2015.rename(columns={'angularjs':'angular.js'})

In [307]:
print_series(df_2015.columns)

0 :  age
1 :  gender
2 :  experience
3 :  salary
4 :  android
5 :  arduino
6 :  angular.js
7 :  c
8 :  c++
9 :  c#
10 :  cassandra
11 :  coffeescript
12 :  cordova
13 :  clojure
14 :  cloud
15 :  dart
16 :  f#
17 :  go
18 :  hadoop
19 :  haskell
20 :  ios
21 :  java
22 :  javascript
23 :  lamp
24 :  matlab
25 :  mongodb
26 :  node.js
27 :  objective-c
28 :  perl
29 :  php
30 :  python
31 :  r
32 :  redis
33 :  ruby
34 :  rust
35 :  salesforce
36 :  scala
37 :  sharepoint
38 :  spark
39 :  sql
40 :  sql server
41 :  swift
42 :  visual basic
43 :  windows phone
44 :  wordpress


In [308]:
# insert year marker for data
df_2015.insert(0,'year',2015)

In [309]:
# convert salary to ordered slabs
df_2015.salary = categorize_salary_range(df_2015.salary)

In [310]:
# check salary values
print_salary_values(df_2015.salary)

,mean,count
"$120,000 - $140,000",0.021544,562
"More than $140,000",0.028444,742
"$100,000 - $120,000",0.037990,991
"$80,000 - $100,000",0.053439,1394
Student / Unemployed,0.076516,1996
"$60,000 - $80,000",0.076938,2007
"$40,000 - $60,000",0.093115,2429
"$20,000 - $40,000",0.104731,2732
"Less than $20,000",0.153339,4000


In [311]:
df_2015.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26086 entries, 0 to 26085
Data columns (total 46 columns):
year             26086 non-null int64
age              25831 non-null object
gender           25744 non-null object
experience       24827 non-null object
salary           16853 non-null category
android          26086 non-null int64
arduino          26086 non-null int64
angular.js       26086 non-null int64
c                26086 non-null int64
c++              26086 non-null int64
c#               26086 non-null int64
cassandra        26086 non-null int64
coffeescript     26086 non-null int64
cordova          26086 non-null int64
clojure          26086 non-null int64
cloud            26086 non-null int64
dart             26086 non-null int64
f#               26086 non-null int64
go               26086 non-null int64
hadoop           26086 non-null int64
haskell          26086 non-null int64
ios              26086 non-null int64
java             26086 non-null int64
javascript 

In [312]:
df_2015.describe()

,year,android,arduino,angular.js,c,c++,c#,cassandra,coffeescript,cordova,...,salesforce,scala,sharepoint,spark,sql,sql server,swift,visual basic,windows phone,wordpress
count,26086.0,26086.000000,26086.000000,26086.000000,26086.000000,26086.000000,26086.000000,26086.000000,26086.000000,26086.000000,...,26086.000000,26086.000000,26086.000000,26086.000000,26086.000000,26086.000000,26086.000000,26086.000000,26086.000000,26086.000000
mean,2015.0,0.157556,0.062332,0.111669,0.138465,0.185540,0.266388,0.007744,0.030016,0.024074,...,0.005865,0.020624,0.013379,0.003987,0.361842,0.158284,0.029096,0.065207,0.021851,0.076938
std,0.0,0.364331,0.241763,0.314965,0.345394,0.388743,0.442078,0.087658,0.170635,0.153283,...,0.076361,0.142125,0.114893,0.063016,0.480543,0.365014,0.168079,0.246896,0.146199,0.266498
min,2015.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2015.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2015.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2015.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2015.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### Read and process 2016 data

In [313]:
df_2016 = pd.read_csv('2016_results.csv')
df_2016.head()

,Unnamed: 0,collector,country,un_subregion,so_region,age_range,age_midpoint,gender,self_identification,occupation,...,important_sameend,important_newtech,important_buildnew,important_buildexisting,important_promotion,important_companymission,important_wfh,important_ownoffice,developer_challenges,why_stack_overflow
0,1888,Facebook,Afghanistan,Southern Asia,Central Asia,20-24,22.0,Male,Programmer,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4637,Facebook,Afghanistan,Southern Asia,Central Asia,30-34,32.0,Male,Developer; Engineer; Programmer,Mobile developer - iOS,...,This is somewhat important,This is very important,This is very important,I don't care about this,This is somewhat important,This is somewhat important,I don't care about this,I don't care about this,Unrealistic expectations; Interacting with non...,To get help for my job; Because I can't do my ...
2,11164,Facebook,Afghanistan,Southern Asia,Central Asia,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,21378,Facebook,Afghanistan,Southern Asia,Central Asia,NaN,NaN,Female,Engineer,DevOps,...,I don't care about this,This is somewhat important,This is very important,This is very important,I don't care about this,I don't care about this,This is very important,This is somewhat important,Corporate policies; Interacting with non-techn...,To get help for my job; Beacause I love to learn
4,30280,Facebook,Afghanistan,Southern Asia,Central Asia,> 60,65.0,Prefer not to disclose,Developer; Engineer; Programmer; Sr. Developer...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [314]:
print_series(df_2016.columns)

0 :  Unnamed: 0
1 :  collector
2 :  country
3 :  un_subregion
4 :  so_region
5 :  age_range
6 :  age_midpoint
7 :  gender
8 :  self_identification
9 :  occupation
10 :  occupation_group
11 :  experience_range
12 :  experience_midpoint
13 :  salary_range
14 :  salary_midpoint
15 :  big_mac_index
16 :  tech_do
17 :  tech_want
18 :  aliens
19 :  programming_ability
20 :  employment_status
21 :  industry
22 :  company_size_range
23 :  team_size_range
24 :  women_on_team
25 :  remote
26 :  job_satisfaction
27 :  job_discovery
28 :  dev_environment
29 :  commit_frequency
30 :  hobby
31 :  dogs_vs_cats
32 :  desktop_os
33 :  unit_testing
34 :  rep_range
35 :  visit_frequency
36 :  why_learn_new_tech
37 :  education
38 :  open_to_new_job
39 :  new_job_value
40 :  job_search_annoyance
41 :  interview_likelihood
42 :  how_to_improve_interview_process
43 :  star_wars_vs_star_trek
44 :  agree_tech
45 :  agree_notice
46 :  agree_problemsolving
47 :  agree_diversity
48 :  agree_adblocker
49 :  agree

In [315]:
# extract the columns we are interested in and the tech cols
df_2016 = clean_filter_cols(df_2016, [5,7,11,13,14], ['age', 'gender', 'experience', 'salary_range', 'salary'], [16], False,'; ')
df_2016.head()

,age,gender,experience,salary_range,salary,android,angularjs,arduino / raspberry pi,c,c#,...,sql server,salesforce,scala,sharepoint,spark,swift,visual basic,windows phone,wordpress,ios
0,20-24,Male,NaN,NaN,NaN,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,30-34,Male,6 - 10 years,"$40,000 - $50,000",45000.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,NaN,Female,11+ years,"Less than $10,000",5000.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,> 60,Prefer not to disclose,NaN,NaN,NaN,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [316]:
# consistent column names
df_2016 = df_2016.rename(columns={'cloud (aws, gae, azure, etc.)':'cloud', 'angularjs':'angular.js', 
                                  'arduino / raspberry pi':'arduino', 'reactjs':'react.js'})

In [317]:
print_series(df_2016.columns)

0 :  age
1 :  gender
2 :  experience
3 :  salary_range
4 :  salary
5 :  android
6 :  angular.js
7 :  arduino
8 :  c
9 :  c#
10 :  c++
11 :  cassandra
12 :  clojure
13 :  cloud
14 :  coffeescript
15 :  cordova
16 :  dart
17 :  f#
18 :  go
19 :  hadoop
20 :  haskell
21 :  java
22 :  javascript
23 :  lamp
24 :  matlab
25 :  mongodb
26 :  node.js
27 :  objective-c
28 :  php
29 :  perl
30 :  python
31 :  r
32 :  react.js
33 :  redis
34 :  ruby
35 :  rust
36 :  sql
37 :  sql server
38 :  salesforce
39 :  scala
40 :  sharepoint
41 :  spark
42 :  swift
43 :  visual basic
44 :  windows phone
45 :  wordpress
46 :  ios


In [318]:
# insert year marker for data
df_2016.insert(0,'year',2016)

In [319]:
# understand salary distribution
df_2016.salary.describe()

count     41742.000000
mean      53952.973025
std       46518.823875
min        5000.000000
25%       15000.000000
50%       45000.000000
75%       75000.000000
max      210000.000000
Name: salary, dtype: float64

In [320]:
# convert salary to bins
df_2016.salary = get_salary_bins(df_2016.salary)

In [321]:
# convert salary to ordered slabs
df_2016.salary = categorize_salary_range(df_2016.salary)

In [322]:
# put back the unemployed / student category
df_2016.salary = df_2016.salary.where(df_2016.salary_range != 'Unemployed', 'Student / Unemployed')
df_2016 = df_2016.drop(columns=['salary_range'])

In [323]:
# check salary values
print_salary_values(df_2016.salary)

,mean,count
"$120,000 - $140,000",0.027218,1525
Student / Unemployed,0.033553,1880
"More than $140,000",0.043530,2439
"$100,000 - $120,000",0.048885,2739
"$80,000 - $100,000",0.062788,3518
"$60,000 - $80,000",0.095003,5323
"$40,000 - $60,000",0.114742,6429
"$20,000 - $40,000",0.126825,7106
"Less than $20,000",0.226004,12663


In [324]:
df_2016.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56030 entries, 0 to 56029
Data columns (total 47 columns):
year             56030 non-null int64
age              55727 non-null object
gender           55586 non-null object
experience       49520 non-null object
salary           43622 non-null category
android          56030 non-null int64
angular.js       56030 non-null int64
arduino          56030 non-null int64
c                56030 non-null int64
c#               56030 non-null int64
c++              56030 non-null int64
cassandra        56030 non-null int64
clojure          56030 non-null int64
cloud            56030 non-null int64
coffeescript     56030 non-null int64
cordova          56030 non-null int64
dart             56030 non-null int64
f#               56030 non-null int64
go               56030 non-null int64
hadoop           56030 non-null int64
haskell          56030 non-null int64
java             56030 non-null int64
javascript       56030 non-null int64
lamp       

In [325]:
df_2016.describe()

,year,android,angular.js,arduino,c,c#,c++,cassandra,clojure,cloud,...,sql server,salesforce,scala,sharepoint,spark,swift,visual basic,windows phone,wordpress,ios
count,56030.0,56030.000000,56030.000000,56030.000000,56030.000000,56030.000000,56030.000000,56030.000000,56030.000000,56030.000000,...,56030.000000,56030.000000,56030.000000,56030.000000,56030.000000,56030.000000,56030.000000,56030.000000,56030.000000,56030.000000
mean,2016.0,0.153507,0.157469,0.067767,0.137034,0.272765,0.171140,0.011833,0.009923,0.082616,...,0.166090,0.011869,0.028592,0.015224,0.012618,0.049009,0.059111,0.020043,0.080243,0.080278
std,0.0,0.360479,0.364246,0.251348,0.343886,0.445385,0.376635,0.108135,0.099121,0.275304,...,0.372164,0.108296,0.166658,0.122444,0.111621,0.215890,0.235835,0.140148,0.271671,0.271726
min,2016.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2016.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2016.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2016.0,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2016.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### Read and process 2017 data


In [326]:
df_2017 = pd.read_csv('2017_results.csv')
df_2017.head()

,Respondent,Professional,ProgramHobby,Country,University,EmploymentStatus,FormalEducation,MajorUndergrad,HomeRemote,CompanySize,...,StackOverflowMakeMoney,Gender,HighestEducationParents,Race,SurveyLong,QuestionsInteresting,QuestionsConfusing,InterestedAnswers,Salary,ExpectedSalary
0,1,Student,"Yes, both",United States,No,"Not employed, and not looking for work",Secondary school,NaN,NaN,NaN,...,Strongly disagree,Male,High school,White or of European descent,Strongly disagree,Strongly agree,Disagree,Strongly agree,NaN,NaN
1,2,Student,"Yes, both",United Kingdom,"Yes, full-time",Employed part-time,Some college/university study without earning ...,Computer science or software engineering,"More than half, but not all, the time",20 to 99 employees,...,Strongly disagree,Male,A master's degree,White or of European descent,Somewhat agree,Somewhat agree,Disagree,Strongly agree,NaN,37500.0
2,3,Professional developer,"Yes, both",United Kingdom,No,Employed full-time,Bachelor's degree,Computer science or software engineering,"Less than half the time, but at least one day ...","10,000 or more employees",...,Disagree,Male,A professional degree,White or of European descent,Somewhat agree,Agree,Disagree,Agree,113750.0,NaN
3,4,Professional non-developer who sometimes write...,"Yes, both",United States,No,Employed full-time,Doctoral degree,A non-computer-focused engineering discipline,"Less than half the time, but at least one day ...","10,000 or more employees",...,Disagree,Male,A doctoral degree,White or of European descent,Agree,Agree,Somewhat agree,Strongly agree,NaN,NaN
4,5,Professional developer,"Yes, I program as a hobby",Switzerland,No,Employed full-time,Master's degree,Computer science or software engineering,Never,10 to 19 employees,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [327]:
print_series(df_2017.columns)

0 :  Respondent
1 :  Professional
2 :  ProgramHobby
3 :  Country
4 :  University
5 :  EmploymentStatus
6 :  FormalEducation
7 :  MajorUndergrad
8 :  HomeRemote
9 :  CompanySize
10 :  CompanyType
11 :  YearsProgram
12 :  YearsCodedJob
13 :  YearsCodedJobPast
14 :  DeveloperType
15 :  WebDeveloperType
16 :  MobileDeveloperType
17 :  NonDeveloperType
18 :  CareerSatisfaction
19 :  JobSatisfaction
20 :  ExCoderReturn
21 :  ExCoderNotForMe
22 :  ExCoderBalance
23 :  ExCoder10Years
24 :  ExCoderBelonged
25 :  ExCoderSkills
26 :  ExCoderWillNotCode
27 :  ExCoderActive
28 :  PronounceGIF
29 :  ProblemSolving
30 :  BuildingThings
31 :  LearningNewTech
32 :  BoringDetails
33 :  JobSecurity
34 :  DiversityImportant
35 :  AnnoyingUI
36 :  FriendsDevelopers
37 :  RightWrongWay
38 :  UnderstandComputers
39 :  SeriousWork
40 :  InvestTimeTools
41 :  WorkPayCare
42 :  KinshipDevelopers
43 :  ChallengeMyself
44 :  CompetePeers
45 :  ChangeWorld
46 :  JobSeekingStatus
47 :  HoursPerWeek
48 :  LastNewJob

In [328]:
df_2017.iloc[:,1].value_counts()

Professional developer                                  36131
Student                                                  8224
Professional non-developer who sometimes writes code     5140
Used to be a professional developer                       983
None of these                                             914
Name: Professional, dtype: int64

In [329]:
df_2017.iloc[:,5].value_counts()

Employed full-time                                      36148
Independent contractor, freelancer, or self-employed     5233
Employed part-time                                       3180
Not employed, and not looking for work                   2791
Not employed, but looking for work                       2786
I prefer not to say                                      1086
Retired                                                   168
Name: EmploymentStatus, dtype: int64

In [330]:
# extract the columns we are interested in and the tech cols
df_2017 = clean_filter_cols(df_2017, [1,5,12,145,152], ['profession','employment','experience', 'gender', 'salary'], [88,90], False, '; ')
df_2017.head()

,profession,employment,experience,gender,salary,assembly,c,c#,c++,clojure,...,visual basic 6,.net core,angularjs,cordova,firebase,hadoop,node.js,react,spark,xamarin
0,Student,"Not employed, and not looking for work",NaN,Male,NaN,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Student,Employed part-time,NaN,Male,NaN,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,Professional developer,Employed full-time,20 or more years,Male,113750.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Professional non-developer who sometimes write...,Employed full-time,9 to 10 years,Male,NaN,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,Professional developer,Employed full-time,10 to 11 years,NaN,NaN,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [331]:
# consistent column names
df_2017 = df_2017.rename(columns={'angularjs':'angular.js','.net core':'.net','react':'react.js', 
                                  'common lisp':'lisp', 'visual basic 6':'visual basic'})

In [332]:
print_series(df_2017.columns)

0 :  profession
1 :  employment
2 :  experience
3 :  gender
4 :  salary
5 :  assembly
6 :  c
7 :  c#
8 :  c++
9 :  clojure
10 :  coffeescript
11 :  lisp
12 :  dart
13 :  elixir
14 :  erlang
15 :  f#
16 :  go
17 :  groovy
18 :  hack
19 :  haskell
20 :  java
21 :  javascript
22 :  julia
23 :  lua
24 :  matlab
25 :  objective-c
26 :  php
27 :  perl
28 :  python
29 :  r
30 :  ruby
31 :  rust
32 :  sql
33 :  scala
34 :  smalltalk
35 :  swift
36 :  typescript
37 :  vb.net
38 :  vba
39 :  visual basic
40 :  .net
41 :  angular.js
42 :  cordova
43 :  firebase
44 :  hadoop
45 :  node.js
46 :  react.js
47 :  spark
48 :  xamarin


In [333]:
# insert year marker for data
df_2017.insert(0,'year',2017)

In [334]:
# understand salary distribution
df_2017.salary.describe()

count     12891.000000
mean      56298.480641
std       39880.905277
min           0.000000
25%       26440.371839
50%       50000.000000
75%       80000.000000
max      197000.000000
Name: salary, dtype: float64

In [335]:
# convert salaries to bins
df_2017.salary = get_salary_bins(df_2017.salary)

In [336]:
# convert salary to ordered slabs
df_2017.salary = categorize_salary_range(df_2017.salary)

In [337]:
# put back unemployment status 
unemployed_list = ['Not employed, and not looking for work', 'Not employed, but looking for work', 'Retired']
df_2017.salary = df_2017.salary.where(~df_2017.employment.isin(unemployed_list), 'Student / Unemployed')
df_2017.salary = df_2017.salary.where(df_2017.profession != 'Student', 'Student / Unemployed')

df_2017 = df_2017.drop(columns=['profession', 'employment'])

In [338]:
# check salary values
print_salary_values(df_2017.salary)

,mean,count
"More than $140,000",0.008717,448
"$120,000 - $140,000",0.010021,515
"$100,000 - $120,000",0.018135,932
"$80,000 - $100,000",0.024245,1246
"$60,000 - $80,000",0.037613,1933
"$20,000 - $40,000",0.046875,2409
"Less than $20,000",0.050611,2601
"$40,000 - $60,000",0.054503,2801
Student / Unemployed,0.189037,9715


In [339]:
df_2017.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51392 entries, 0 to 51391
Data columns (total 48 columns):
year            51392 non-null int64
experience      40890 non-null object
gender          35047 non-null object
salary          22600 non-null category
assembly        51392 non-null int64
c               51392 non-null int64
c#              51392 non-null int64
c++             51392 non-null int64
clojure         51392 non-null int64
coffeescript    51392 non-null int64
lisp            51392 non-null int64
dart            51392 non-null int64
elixir          51392 non-null int64
erlang          51392 non-null int64
f#              51392 non-null int64
go              51392 non-null int64
groovy          51392 non-null int64
hack            51392 non-null int64
haskell         51392 non-null int64
java            51392 non-null int64
javascript      51392 non-null int64
julia           51392 non-null int64
lua             51392 non-null int64
matlab          51392 non-null int6

In [340]:
df_2017.describe()

,year,assembly,c,c#,c++,clojure,coffeescript,lisp,dart,elixir,...,visual basic,.net,angular.js,cordova,firebase,hadoop,node.js,react.js,spark,xamarin
count,51392.0,51392.000000,51392.000000,51392.000000,51392.000000,51392.000000,51392.000000,51392.000000,51392.000000,51392.000000,...,51392.000000,51392.000000,51392.000000,51392.000000,51392.000000,51392.000000,51392.000000,51392.000000,51392.000000,51392.000000
mean,2017.0,0.035472,0.135702,0.242762,0.158682,0.007608,0.023194,0.005312,0.002821,0.007394,...,0.020840,0.131596,0.174385,0.043431,0.038508,0.022338,0.185282,0.076919,0.018544,0.032593
std,0.0,0.184972,0.342475,0.428756,0.365383,0.086893,0.150522,0.072691,0.053043,0.085672,...,0.142849,0.338055,0.379444,0.203827,0.192421,0.147782,0.388530,0.266465,0.134908,0.177570
min,2017.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2017.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2017.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2017.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2017.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### Read and process 2018 data

In [341]:
df_2018 = pd.read_csv('2018_results.csv', low_memory=False)
df_2018.head()

,Respondent,Hobby,OpenSource,Country,Student,Employment,FormalEducation,UndergradMajor,CompanySize,DevType,...,Exercise,Gender,SexualOrientation,EducationParents,RaceEthnicity,Age,Dependents,MilitaryUS,SurveyTooLong,SurveyEasy
0,1,Yes,No,Kenya,No,Employed part-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)",Mathematics or statistics,20 to 99 employees,Full-stack developer,...,3 - 4 times per week,Male,Straight or heterosexual,"Bachelor’s degree (BA, BS, B.Eng., etc.)",Black or of African descent,25 - 34 years old,Yes,NaN,The survey was an appropriate length,Very easy
1,3,Yes,Yes,United Kingdom,No,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)","A natural science (ex. biology, chemistry, phy...","10,000 or more employees",Database administrator;DevOps specialist;Full-...,...,Daily or almost every day,Male,Straight or heterosexual,"Bachelor’s degree (BA, BS, B.Eng., etc.)",White or of European descent,35 - 44 years old,Yes,NaN,The survey was an appropriate length,Somewhat easy
2,4,Yes,Yes,United States,No,Employed full-time,Associate degree,"Computer science, computer engineering, or sof...",20 to 99 employees,Engineering manager;Full-stack developer,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5,No,No,United States,No,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",100 to 499 employees,Full-stack developer,...,I don't typically exercise,Male,Straight or heterosexual,Some college/university study without earning ...,White or of European descent,35 - 44 years old,No,No,The survey was an appropriate length,Somewhat easy
4,7,Yes,No,South Africa,"Yes, part-time",Employed full-time,Some college/university study without earning ...,"Computer science, computer engineering, or sof...","10,000 or more employees",Data or business analyst;Desktop or enterprise...,...,3 - 4 times per week,Male,Straight or heterosexual,Some college/university study without earning ...,White or of European descent,18 - 24 years old,Yes,NaN,The survey was an appropriate length,Somewhat easy


In [342]:
print_series(df_2018.columns)

0 :  Respondent
1 :  Hobby
2 :  OpenSource
3 :  Country
4 :  Student
5 :  Employment
6 :  FormalEducation
7 :  UndergradMajor
8 :  CompanySize
9 :  DevType
10 :  YearsCoding
11 :  YearsCodingProf
12 :  JobSatisfaction
13 :  CareerSatisfaction
14 :  HopeFiveYears
15 :  JobSearchStatus
16 :  LastNewJob
17 :  AssessJob1
18 :  AssessJob2
19 :  AssessJob3
20 :  AssessJob4
21 :  AssessJob5
22 :  AssessJob6
23 :  AssessJob7
24 :  AssessJob8
25 :  AssessJob9
26 :  AssessJob10
27 :  AssessBenefits1
28 :  AssessBenefits2
29 :  AssessBenefits3
30 :  AssessBenefits4
31 :  AssessBenefits5
32 :  AssessBenefits6
33 :  AssessBenefits7
34 :  AssessBenefits8
35 :  AssessBenefits9
36 :  AssessBenefits10
37 :  AssessBenefits11
38 :  JobContactPriorities1
39 :  JobContactPriorities2
40 :  JobContactPriorities3
41 :  JobContactPriorities4
42 :  JobContactPriorities5
43 :  JobEmailPriorities1
44 :  JobEmailPriorities2
45 :  JobEmailPriorities3
46 :  JobEmailPriorities4
47 :  JobEmailPriorities5
48 :  JobEmai

In [343]:
df_2018.iloc[:,4].value_counts()

No                70399
Yes, full-time    18394
Yes, part-time     6108
Name: Student, dtype: int64

In [344]:
df_2018.iloc[:,5].value_counts()

Employed full-time                                      70495
Independent contractor, freelancer, or self-employed     9282
Not employed, but looking for work                       5805
Employed part-time                                       5380
Not employed, and not looking for work                   4132
Retired                                                   227
Name: Employment, dtype: int64

In [345]:
df_2018.iloc[:,65]

0                               JavaScript;Python;HTML;CSS
1                             JavaScript;Python;Bash/Shell
2                                                      NaN
3         C#;JavaScript;SQL;TypeScript;HTML;CSS;Bash/Shell
4                       C;C++;Java;Matlab;R;SQL;Bash/Shell
5               Java;JavaScript;Python;TypeScript;HTML;CSS
6                                      JavaScript;HTML;CSS
7                           JavaScript;TypeScript;HTML;CSS
8        Assembly;CoffeeScript;Erlang;Go;JavaScript;Lua...
9                                                      NaN
10                                                     NaN
11                                                     NaN
12                                                     NaN
13                                                    Java
14                     Java;JavaScript;PHP;VB.NET;HTML;CSS
15                                                     NaN
16                                                     N

In [346]:
df_2018.iloc[:,71]

0                      Django;React
1                            Django
2                               NaN
3                               NaN
4                               NaN
5                   Angular;Node.js
6                     Node.js;React
7                   Angular;Node.js
8        Hadoop;Node.js;React;Spark
9                               NaN
10                              NaN
11                              NaN
12                              NaN
13                           Spring
14                              NaN
15                              NaN
16                              NaN
17                        .NET Core
18                          Angular
19                              NaN
20                  Angular;Node.js
21                           Django
22                        .NET Core
23                              NaN
24                  Angular;Cordova
25                  Angular;Node.js
26                    Node.js;React
27                          

In [347]:
# extract the columns we are interested in and the tech cols
df_2018 = clean_filter_cols(df_2018, [4,5,11,54,120,124], ['student','employment','experience', 'salary','gender', 'age'], [65,71], False, ';')
df_2018.head()

,student,employment,experience,salary,gender,age,assembly,bash/shell,c,c#,...,cordova,django,hadoop,node.js,react,spark,spring,tensorflow,torch/pytorch,xamarin
0,No,Employed part-time,3-5 years,NaN,Male,25 - 34 years old,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
1,No,Employed full-time,18-20 years,70841.0,Male,35 - 44 years old,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
2,No,Employed full-time,6-8 years,NaN,NaN,NaN,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,No,Employed full-time,12-14 years,NaN,Male,35 - 44 years old,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,"Yes, part-time",Employed full-time,0-2 years,21426.0,Male,18 - 24 years old,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [348]:
# consistent column names
df_2018 = df_2018.rename(columns={'bash/shell': 'bash', '.net core': '.net', 'angular':'angular.js', 
                                  'react':'react.js', 'delphi/object pascal':'delphi', 'visual basic 6':'visual basic',
                                  'torch/pytorch':'pytorch'})

In [349]:
print_series(df_2018.columns)

0 :  student
1 :  employment
2 :  experience
3 :  salary
4 :  gender
5 :  age
6 :  assembly
7 :  bash
8 :  c
9 :  c#
10 :  c++
11 :  css
12 :  clojure
13 :  cobol
14 :  coffeescript
15 :  delphi
16 :  erlang
17 :  f#
18 :  go
19 :  groovy
20 :  html
21 :  hack
22 :  haskell
23 :  java
24 :  javascript
25 :  julia
26 :  kotlin
27 :  lua
28 :  matlab
29 :  objective-c
30 :  ocaml
31 :  php
32 :  perl
33 :  python
34 :  r
35 :  ruby
36 :  rust
37 :  sql
38 :  scala
39 :  swift
40 :  typescript
41 :  vb.net
42 :  vba
43 :  visual basic
44 :  .net
45 :  angular.js
46 :  cordova
47 :  django
48 :  hadoop
49 :  node.js
50 :  react.js
51 :  spark
52 :  spring
53 :  tensorflow
54 :  pytorch
55 :  xamarin


In [350]:
# insert year marker into data
df_2018.insert(0,'year',2018)

In [351]:
# understand salary distribution
df_2018.salary.describe()

count    4.770200e+04
mean     9.578086e+04
std      2.023482e+05
min      0.000000e+00
25%      2.384400e+04
50%      5.507500e+04
75%      9.300000e+04
max      2.000000e+06
Name: salary, dtype: float64

In [352]:
# convert salaries to bins
df_2018.salary = get_salary_bins(df_2018.salary)

In [353]:
# convert salary to ordered slabs
df_2018.salary = categorize_salary_range(df_2018.salary)

In [354]:
# put back unemployed status
unemployed_list = ['Not employed, and not looking for work', 'Not employed, but looking for work', 'Retired']
df_2018.salary = df_2018.salary.where(~df_2018.employment.isin(unemployed_list), 'Student / Unemployed')
df_2018.salary = df_2018.salary.where(df_2018.student != 'Yes, full-time', 'Student / Unemployed')

df_2018 = df_2018.drop(columns=['student', 'employment'])

In [355]:
# check salary values
print_salary_values(df_2018.salary)

,mean,count
"$120,000 - $140,000",0.020029,1980
"$100,000 - $120,000",0.031511,3115
"$80,000 - $100,000",0.044773,4426
"More than $140,000",0.045157,4464
"$20,000 - $40,000",0.063548,6282
"$60,000 - $80,000",0.066754,6599
"Less than $20,000",0.068889,6810
"$40,000 - $60,000",0.072156,7133
Student / Unemployed,0.237469,23475


In [356]:
df_2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98855 entries, 0 to 98854
Data columns (total 55 columns):
year            98855 non-null int64
experience      77903 non-null object
salary          64284 non-null category
gender          64469 non-null object
age             64574 non-null object
assembly        98855 non-null int64
bash            98855 non-null int64
c               98855 non-null int64
c#              98855 non-null int64
c++             98855 non-null int64
css             98855 non-null int64
clojure         98855 non-null int64
cobol           98855 non-null int64
coffeescript    98855 non-null int64
delphi          98855 non-null int64
erlang          98855 non-null int64
f#              98855 non-null int64
go              98855 non-null int64
groovy          98855 non-null int64
html            98855 non-null int64
hack            98855 non-null int64
haskell         98855 non-null int64
java            98855 non-null int64
javascript      98855 non-null int

In [357]:
df_2018.describe()

,year,assembly,bash,c,c#,c++,css,clojure,cobol,coffeescript,...,cordova,django,hadoop,node.js,react.js,spark,spring,tensorflow,pytorch,xamarin
count,98855.0,98855.000000,98855.000000,98855.000000,98855.000000,98855.000000,98855.000000,98855.00000,98855.000000,98855.000000,...,98855.000000,98855.000000,98855.000000,98855.000000,98855.000000,98855.000000,98855.000000,98855.000000,98855.000000,98855.00000
mean,2018.0,0.058267,0.315331,0.182510,0.272662,0.201022,0.515695,0.01044,0.005968,0.026331,...,0.044196,0.068009,0.024753,0.258945,0.145031,0.025037,0.091842,0.040726,0.008781,0.03840
std,0.0,0.234249,0.464650,0.386266,0.445331,0.400766,0.499756,0.10164,0.077025,0.160120,...,0.205531,0.251762,0.155374,0.438058,0.352133,0.156237,0.288804,0.197656,0.093293,0.19216
min,2018.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
25%,2018.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
50%,2018.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
75%,2018.0,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
max,2018.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000


### Read and process 2019 data

In [358]:
df_2019 = pd.read_csv('2019_results.csv')
df_2019.head()

,Respondent,MainBranch,Hobbyist,OpenSourcer,OpenSource,Employment,Country,Student,EdLevel,UndergradMajor,...,WelcomeChange,SONewContent,Age,Gender,Trans,Sexuality,Ethnicity,Dependents,SurveyLength,SurveyEase
0,1,I am a student who is learning to code,Yes,Never,The quality of OSS and closed source software ...,"Not employed, and not looking for work",United Kingdom,No,Primary/elementary school,NaN,...,Just as welcome now as I felt last year,Tech articles written by other developers;Indu...,14.0,Man,No,Straight / Heterosexual,NaN,No,Appropriate in length,Neither easy nor difficult
1,2,I am a student who is learning to code,No,Less than once per year,The quality of OSS and closed source software ...,"Not employed, but looking for work",Bosnia and Herzegovina,"Yes, full-time","Secondary school (e.g. American high school, G...",NaN,...,Just as welcome now as I felt last year,Tech articles written by other developers;Indu...,19.0,Man,No,Straight / Heterosexual,NaN,No,Appropriate in length,Neither easy nor difficult
2,3,"I am not primarily a developer, but I write co...",Yes,Never,The quality of OSS and closed source software ...,Employed full-time,Thailand,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)",Web development or web design,...,Just as welcome now as I felt last year,Tech meetups or events in your area;Courses on...,28.0,Man,No,Straight / Heterosexual,NaN,Yes,Appropriate in length,Neither easy nor difficult
3,4,I am a developer by profession,No,Never,The quality of OSS and closed source software ...,Employed full-time,United States,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",...,Just as welcome now as I felt last year,Tech articles written by other developers;Indu...,22.0,Man,No,Straight / Heterosexual,White or of European descent,No,Appropriate in length,Easy
4,5,I am a developer by profession,Yes,Once a month or more often,"OSS is, on average, of HIGHER quality than pro...",Employed full-time,Ukraine,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",...,Just as welcome now as I felt last year,Tech meetups or events in your area;Courses on...,30.0,Man,No,Straight / Heterosexual,White or of European descent;Multiracial,No,Appropriate in length,Easy


In [359]:
print_series(df_2019.columns)

0 :  Respondent
1 :  MainBranch
2 :  Hobbyist
3 :  OpenSourcer
4 :  OpenSource
5 :  Employment
6 :  Country
7 :  Student
8 :  EdLevel
9 :  UndergradMajor
10 :  EduOther
11 :  OrgSize
12 :  DevType
13 :  YearsCode
14 :  Age1stCode
15 :  YearsCodePro
16 :  CareerSat
17 :  JobSat
18 :  MgrIdiot
19 :  MgrMoney
20 :  MgrWant
21 :  JobSeek
22 :  LastHireDate
23 :  LastInt
24 :  FizzBuzz
25 :  JobFactors
26 :  ResumeUpdate
27 :  CurrencySymbol
28 :  CurrencyDesc
29 :  CompTotal
30 :  CompFreq
31 :  ConvertedComp
32 :  WorkWeekHrs
33 :  WorkPlan
34 :  WorkChallenge
35 :  WorkRemote
36 :  WorkLoc
37 :  ImpSyn
38 :  CodeRev
39 :  CodeRevHrs
40 :  UnitTests
41 :  PurchaseHow
42 :  PurchaseWhat
43 :  LanguageWorkedWith
44 :  LanguageDesireNextYear
45 :  DatabaseWorkedWith
46 :  DatabaseDesireNextYear
47 :  PlatformWorkedWith
48 :  PlatformDesireNextYear
49 :  WebFrameWorkedWith
50 :  WebFrameDesireNextYear
51 :  MiscTechWorkedWith
52 :  MiscTechDesireNextYear
53 :  DevEnviron
54 :  OpSys
55 :  Con

In [360]:
df_2019.iloc[:,7].value_counts()

No                65816
Yes, full-time    15769
Yes, part-time     5429
Name: Student, dtype: int64

In [361]:
df_2019.iloc[:,5].value_counts()

Employed full-time                                      64440
Independent contractor, freelancer, or self-employed     8511
Not employed, but looking for work                       5600
Employed part-time                                       4469
Not employed, and not looking for work                   3803
Retired                                                   358
Name: Employment, dtype: int64

In [362]:
df_2019.iloc[:,51]

0                                      Node.js
1                                          NaN
2                                          NaN
3                                         .NET
4                              Cordova;Node.js
5                                       Hadoop
6                                          NaN
7                        Hadoop;Node.js;Pandas
8                             Node.js;Unity 3D
9            .NET;Node.js;Pandas;Torch/PyTorch
10                                         NaN
11       Ansible;Chef;Hadoop;Pandas;TensorFlow
12                             Node.js;Xamarin
13                                         NaN
14                                         NaN
15                      .NET;.NET Core;Node.js
16                                     Node.js
17                                         NaN
18                      .NET;.NET Core;Node.js
19                                 .NET;Pandas
20                                         NaN
21           

In [363]:
# extract the columns we are interested in and the tech cols
df_2019 = clean_filter_cols(df_2019, [5,7,15,31,77,78], ['employment', 'student', 'experience', 'salary', 'age', 'gender'], [43, 49, 51], False, ';')
df_2019.head()

,employment,student,experience,salary,age,gender,assembly,bash/shell/powershell,c,c#,...,node.js,other(s):,pandas,puppet,react native,tensorflow,torch/pytorch,unity 3d,unreal engine,xamarin
0,"Not employed, and not looking for work",No,NaN,NaN,14.0,Man,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,"Not employed, but looking for work","Yes, full-time",NaN,NaN,19.0,Man,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Employed full-time,No,1,8820.0,28.0,Man,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Employed full-time,No,Less than 1 year,61000.0,22.0,Man,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,Employed full-time,No,9,NaN,30.0,Man,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [364]:
# merge data in similar columns and drop redundant ones
df_2019['.net'] = df_2019['.net'].where(df_2019['.net'] == 1, df_2019['.net core'])
df_2019['ruby'] = df_2019['ruby'].where(df_2019['ruby'] == 1, df_2019['ruby on rails'])
df_2019 = df_2019.drop(columns=['other(s):', 'other(s):', 'other(s):', '.net core', 'ruby on rails'])

In [365]:
# make column names consistent
df_2019 = df_2019.rename(columns={'html/css':'html','bash/shell/powershell':'bash','angular/angular.js':'angular.js',
                                 'apache spark':'spark', 'torch/pytorch':'pytorch'})


In [366]:
print_series(df_2019.columns)

0 :  employment
1 :  student
2 :  experience
3 :  salary
4 :  age
5 :  gender
6 :  assembly
7 :  bash
8 :  c
9 :  c#
10 :  c++
11 :  clojure
12 :  dart
13 :  elixir
14 :  erlang
15 :  f#
16 :  go
17 :  html
18 :  java
19 :  javascript
20 :  kotlin
21 :  objective-c
22 :  php
23 :  python
24 :  r
25 :  ruby
26 :  rust
27 :  sql
28 :  scala
29 :  swift
30 :  typescript
31 :  vba
32 :  webassembly
33 :  asp.net
34 :  angular.js
35 :  django
36 :  drupal
37 :  express
38 :  flask
39 :  laravel
40 :  react.js
41 :  spring
42 :  vue.js
43 :  jquery
44 :  .net
45 :  ansible
46 :  spark
47 :  chef
48 :  cordova
49 :  cryengine
50 :  flutter
51 :  hadoop
52 :  node.js
53 :  pandas
54 :  puppet
55 :  react native
56 :  tensorflow
57 :  pytorch
58 :  unity 3d
59 :  unreal engine
60 :  xamarin


In [367]:
# set year marker in data
df_2019.insert(0,'year',2019)

In [368]:
# understand salary distribution
df_2019.salary.describe()

count    5.582300e+04
mean     1.271107e+05
std      2.841523e+05
min      0.000000e+00
25%      2.577750e+04
50%      5.728700e+04
75%      1.000000e+05
max      2.000000e+06
Name: salary, dtype: float64

In [369]:
# convert salaries to bins
df_2019.salary = get_salary_bins(df_2019.salary)

In [370]:
# convert salary to ordered slabs
df_2019.salary = categorize_salary_range(df_2019.salary)

In [371]:
# put back unemployed status
unemployed_list = ['Not employed, and not looking for work', 'Not employed, but looking for work', 'Retired']
df_2019.salary = df_2019.salary.where(~df_2019.employment.isin(unemployed_list), 'Student / Unemployed')
df_2019.salary = df_2019.salary.where(df_2019.student != 'Yes, full-time', 'Student / Unemployed')

df_2019 = df_2019.drop(columns=['student', 'employment'])

In [372]:
# check salary values
print_salary_values(df_2019.salary)

,mean,count
"$120,000 - $140,000",0.025967,2308
"$100,000 - $120,000",0.037296,3315
"$80,000 - $100,000",0.056434,5016
"$60,000 - $80,000",0.079993,7110
"More than $140,000",0.086046,7648
"$20,000 - $40,000",0.090118,8010
"Less than $20,000",0.096723,8597
"$40,000 - $60,000",0.097116,8632
Student / Unemployed,0.231585,20584


In [373]:
df_2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88883 entries, 0 to 88882
Data columns (total 60 columns):
year             88883 non-null int64
experience       74331 non-null object
salary           71220 non-null category
age              79210 non-null float64
gender           85406 non-null object
assembly         88883 non-null int64
bash             88883 non-null int64
c                88883 non-null int64
c#               88883 non-null int64
c++              88883 non-null int64
clojure          88883 non-null int64
dart             88883 non-null int64
elixir           88883 non-null int64
erlang           88883 non-null int64
f#               88883 non-null int64
go               88883 non-null int64
html             88883 non-null int64
java             88883 non-null int64
javascript       88883 non-null int64
kotlin           88883 non-null int64
objective-c      88883 non-null int64
php              88883 non-null int64
python           88883 non-null int64
r         

In [374]:
df_2019.describe()

,year,age,assembly,bash,c,c#,c++,clojure,dart,elixir,...,hadoop,node.js,pandas,puppet,react native,tensorflow,pytorch,unity 3d,unreal engine,xamarin
count,88883.0,79210.000000,88883.000000,88883.000000,88883.000000,88883.000000,88883.000000,88883.000000,88883.000000,88883.000000,...,88883.000000,88883.000000,88883.000000,88883.000000,88883.000000,88883.000000,88883.000000,88883.000000,88883.000000,88883.000000
mean,2019.0,30.336699,0.065626,0.359923,0.202705,0.304861,0.230910,0.014108,0.018935,0.014176,...,0.032132,0.328601,0.083784,0.017517,0.069001,0.067606,0.021624,0.074559,0.023120,0.042865
std,0.0,9.178390,0.247628,0.479980,0.402017,0.460351,0.421418,0.117939,0.136296,0.118216,...,0.176352,0.469707,0.277066,0.131190,0.253457,0.251070,0.145453,0.262679,0.150286,0.202555
min,2019.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2019.0,24.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2019.0,29.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2019.0,35.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2019.0,99.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## Merge the data from all the years

In [375]:
df_array = [df_2011, df_2012, df_2013, df_2014, df_2015, df_2016, df_2017, df_2018, df_2019]
for df in df_array:
    print(df.shape)

(2813, 16)
(6243, 17)
(9742, 16)
(7643, 16)
(26086, 46)
(56030, 47)
(51392, 48)
(98855, 55)
(88883, 60)


In [376]:
df_comb = pd.concat(df_array, axis=0, ignore_index=True, sort=False)

In [377]:
df_comb.shape

(347687, 90)

In [378]:
# checking that we have all the years
df_comb.year.unique()

array([2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019])

In [379]:
df_comb.columns.sort_values()

Index(['.net', 'age', 'android', 'angular.js', 'ansible', 'arduino', 'asp.net',
       'assembly', 'bash', 'c', 'c#', 'c++', 'cassandra', 'chef', 'clojure',
       'cloud', 'cobol', 'coffeescript', 'cordova', 'cryengine', 'css', 'dart',
       'delphi', 'django', 'drupal', 'elixir', 'erlang', 'experience',
       'express', 'f#', 'firebase', 'flask', 'flutter', 'gender', 'go',
       'groovy', 'hack', 'hadoop', 'haskell', 'html', 'ios', 'java',
       'javascript', 'jquery', 'julia', 'kotlin', 'lamp', 'laravel', 'lisp',
       'lua', 'matlab', 'mongodb', 'node.js', 'objective-c', 'ocaml', 'pandas',
       'perl', 'php', 'puppet', 'python', 'pytorch', 'r', 'react native',
       'react.js', 'redis', 'ruby', 'rust', 'salary', 'salesforce', 'scala',
       'sharepoint', 'smalltalk', 'spark', 'spring', 'sql', 'sql server',
       'swift', 'tensorflow', 'typescript', 'unity 3d', 'unreal engine',
       'vb.net', 'vba', 'visual basic', 'vue.js', 'webassembly',
       'windows phone', 'wordpr

In [380]:
df_comb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 347687 entries, 0 to 347686
Data columns (total 90 columns):
year             347687 non-null int64
age              250815 non-null object
experience       292946 non-null object
gender           273598 non-null object
salary           236899 non-null category
java             347687 non-null int64
javascript       347687 non-null int64
css              107911 non-null float64
php              347687 non-null int64
python           347687 non-null int64
ruby             347687 non-null int64
sql              347687 non-null int64
c#               347687 non-null int64
c++              347687 non-null int64
c                347687 non-null int64
perl             241419 non-null float64
objective-c      344874 non-null float64
html             193981 non-null float64
jquery           98625 non-null float64
node.js          338631 non-null float64
android          82116 non-null float64
arduino          82116 non-null float64
angular.js  

#### Write to file

In [381]:
df_comb.to_csv('code_survey.csv', index=False)